# ============================================================
# 6. MODEL BUILDING & TRAINING (Person B - Phase 5)
# ============================================================

## Phase 5: Data Splitting & Model Architecture Design

**Person B's Tasks:**
- Load preprocessed data from Person A
- Split data into train/validation/test sets
- Design neural network architecture
- Compile model



In [ ]:
# Step 5.1: Load Preprocessed Data from Person A
# ============================================================

# Import Person A's preprocessing function
# (This function should be defined in Section 5 above)
# If not, uncomment and run the function definition from Section 5 first

# Load preprocessed data
X, y, feature_names, scaler = load_and_preprocess()

print("="*60)
print("DATA LOADED FROM PERSON A")
print("="*60)
print(f"✓ Data loaded successfully!")
print(f"  - Features shape: {X.shape}")
print(f"  - Target shape: {y.shape}")
print(f"  - Number of features: {len(feature_names)}")
print(f"  - Feature names: {feature_names[:5]}... (showing first 5)")



In [ ]:
# Step 5.2: Split Data into Train/Validation/Test Sets
# ============================================================

from sklearn.model_selection import train_test_split
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Split into train (70%) and temp (30%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, 
    test_size=0.3, 
    random_state=42, 
    stratify=y  # Maintain class balance
)

# Split temp into validation (15%) and test (15%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,  # 0.5 of 0.3 = 0.15 (15%)
    random_state=42,
    stratify=y_temp
)

print("="*60)
print("DATA SPLITTING COMPLETE")
print("="*60)
print(f"\n✓ Data split completed!")
print(f"  - Training set: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"  - Validation set: {X_val.shape[0]} samples ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"  - Test set: {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

# Verify class distribution
print(f"\n📊 Class Distribution:")
print(f"  Training - Class 0: {(y_train == 0).sum()}, Class 1: {(y_train == 1).sum()}")
print(f"  Validation - Class 0: {(y_val == 0).sum()}, Class 1: {(y_val == 1).sum()}")
print(f"  Test - Class 0: {(y_test == 0).sum()}, Class 1: {(y_test == 1).sum()}")



In [ ]:
# Step 5.3: Design Neural Network Architecture
# ============================================================

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Set random seed for reproducibility
tf.random.set_seed(42)

# Define model architecture
model = keras.Sequential([
    # Input layer
    layers.Dense(64, activation='relu', input_shape=(15,), name='input_layer'),
    layers.Dropout(0.3, name='dropout_1'),
    
    # Hidden layer 1
    layers.Dense(32, activation='relu', name='hidden_layer_1'),
    layers.Dropout(0.3, name='dropout_2'),
    
    # Hidden layer 2
    layers.Dense(16, activation='relu', name='hidden_layer_2'),
    layers.Dropout(0.2, name='dropout_3'),
    
    # Output layer (binary classification)
    layers.Dense(1, activation='sigmoid', name='output_layer')
])

print("="*60)
print("MODEL ARCHITECTURE CREATED")
print("="*60)
print("✓ Model architecture created!")
print("\n📋 Model Summary:")
model.summary()



In [ ]:
# Step 5.4: Compile Model
# ============================================================

# Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',  # Binary classification
    metrics=['accuracy', 'precision', 'recall']
)

print("="*60)
print("MODEL COMPILED")
print("="*60)
print("✓ Model compiled successfully!")
print("  - Optimizer: Adam (lr=0.001)")
print("  - Loss: binary_crossentropy")
print("  - Metrics: accuracy, precision, recall")
print("\n✅ Phase 5 Complete - Model ready for training!")



# ============================================================
# 7. MODEL TRAINING (Person B - Phase 6)
# ============================================================

## Phase 6: Model Training & Early Stopping

**Person B's Tasks:**
- Set up callbacks (early stopping, model checkpointing)
- Train the model
- Visualize training history



In [ ]:
# Step 6.1: Set Up Callbacks
# ============================================================

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import os

# Create models directory if it doesn't exist
os.makedirs('../models', exist_ok=True)

# Define callbacks
callbacks = [
    # Early stopping: stop if validation loss doesn't improve
    EarlyStopping(
        monitor='val_loss',
        patience=15,  # Wait 15 epochs
        restore_best_weights=True,
        verbose=1,
        mode='min'
    ),
    
    # Save best model
    ModelCheckpoint(
        filepath='../models/heart_failure_model_best.h5',
        monitor='val_loss',
        save_best_only=True,
        verbose=1,
        mode='min'
    ),
    
    # Reduce learning rate if plateau
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=0.00001,
        verbose=1,
        mode='min'
    )
]

print("="*60)
print("CALLBACKS CONFIGURED")
print("="*60)
print("✓ Callbacks configured:")
print("  - EarlyStopping: patience=15, monitor='val_loss'")
print("  - ModelCheckpoint: save best model to '../models/heart_failure_model_best.h5'")
print("  - ReduceLROnPlateau: factor=0.5, patience=5")



In [ ]:
# Step 6.2: Train the Model
# ============================================================

# Train the model
print("="*60)
print("TRAINING MODEL")
print("="*60)
print("Starting training...")
print("This may take a few minutes...\n")

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,  # Maximum epochs
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*60)
print("TRAINING COMPLETE")
print("="*60)
print(f"✓ Training completed!")
print(f"  - Total epochs trained: {len(history.history['loss'])}")
print(f"  - Best validation loss: {min(history.history['val_loss']):.4f}")
print(f"  - Best validation accuracy: {max(history.history['val_accuracy']):.4f}")
print(f"  - Model saved to: '../models/heart_failure_model_best.h5'")



In [ ]:
# Step 6.3: Visualize Training History
# ============================================================

import matplotlib.pyplot as plt

# Plot training history
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Loss
axes[0, 0].plot(history.history['loss'], label='Training Loss', linewidth=2)
axes[0, 0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0, 0].set_title('Model Loss', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Accuracy
axes[0, 1].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
axes[0, 1].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[0, 1].set_title('Model Accuracy', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Precision
axes[1, 0].plot(history.history['precision'], label='Training Precision', linewidth=2)
axes[1, 0].plot(history.history['val_precision'], label='Validation Precision', linewidth=2)
axes[1, 0].set_title('Model Precision', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Precision')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Recall
axes[1, 1].plot(history.history['recall'], label='Training Recall', linewidth=2)
axes[1, 1].plot(history.history['val_recall'], label='Validation Recall', linewidth=2)
axes[1, 1].set_title('Model Recall', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Recall')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("="*60)
print("TRAINING HISTORY VISUALIZED")
print("="*60)
print("✓ Training history plots displayed")
print("\n✅ Phase 6 Complete - Model trained and ready for evaluation!")



# ============================================================
# 8. HYPERPARAMETER TUNING & MODEL OPTIMIZATION (Person B - Phase 7)
# ============================================================

## Phase 7: Hyperparameter Tuning & Model Optimization

**Person B's Tasks:**
- Evaluate initial model on test set
- Perform hyperparameter tuning
- Train optimized model
- Evaluate final model
- Save final model for Person C



In [ ]:
# Step 7.1: Evaluate Initial Model
# ============================================================

from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Load best model from Phase 6
best_model = load_model('../models/heart_failure_model_best.h5')

# Evaluate on test set
print("="*60)
print("EVALUATING INITIAL MODEL (Test Set)")
print("="*60)

test_loss, test_accuracy, test_precision, test_recall = best_model.evaluate(
    X_test, y_test, verbose=0
)

print("📊 Initial Model Performance (Test Set):")
print(f"  - Loss: {test_loss:.4f}")
print(f"  - Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"  - Precision: {test_precision:.4f}")
print(f"  - Recall: {test_recall:.4f}")

# Calculate F1 score
f1_score = 2 * (test_precision * test_recall) / (test_precision + test_recall)
print(f"  - F1 Score: {f1_score:.4f}")

# Get predictions
y_pred_proba = best_model.predict(X_test, verbose=0)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("\n📊 Confusion Matrix:")
print(cm)

print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred, target_names=['No Disease', 'Disease']))

print("\n✅ Initial model evaluation complete!")



In [ ]:
# Step 7.2: Hyperparameter Tuning
# ============================================================

print("="*60)
print("HYPERPARAMETER TUNING")
print("="*60)
print("Testing different learning rates...")
print("(This may take a few minutes)\n")

# Try different learning rates
learning_rates = [0.0001, 0.001, 0.01]
tuning_results = []

for lr in learning_rates:
    print(f"\n🔧 Testing learning rate: {lr}")
    
    # Create new model with same architecture
    model_tune = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=(15,)),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(16, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model_tune.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    # Train with fewer epochs for tuning
    early_stop_tune = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=0
    )
    
    history_tune = model_tune.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=30,
        batch_size=32,
        callbacks=[early_stop_tune],
        verbose=0
    )
    
    # Evaluate on validation set
    val_loss, val_acc = model_tune.evaluate(X_val, y_val, verbose=0)
    tuning_results.append({
        'lr': lr,
        'val_loss': val_loss,
        'val_acc': val_acc,
        'epochs': len(history_tune.history['loss'])
    })
    print(f"  ✓ Validation Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}, Epochs: {len(history_tune.history['loss'])}")

# Find best learning rate
best_lr_result = min(tuning_results, key=lambda x: x['val_loss'])
best_lr = best_lr_result['lr']

print("\n" + "="*60)
print("HYPERPARAMETER TUNING RESULTS")
print("="*60)
print("📊 Results Summary:")
for result in tuning_results:
    marker = " ⭐ BEST" if result['lr'] == best_lr else ""
    print(f"  LR {result['lr']:6.4f}: Val Loss={result['val_loss']:.4f}, Val Acc={result['val_acc']:.4f}{marker}")

print(f"\n✓ Best learning rate: {best_lr} (Val Loss: {best_lr_result['val_loss']:.4f})")



In [ ]:
# Step 7.3: Train Final Optimized Model
# ============================================================

print("="*60)
print("TRAINING FINAL OPTIMIZED MODEL")
print("="*60)
print(f"Using best learning rate: {best_lr}")
print("Training with full epochs...\n")

# Create optimized model with best hyperparameters
final_model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(15,)),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(16, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')
])

final_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=best_lr),
    loss='binary_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)

# Train final model with full callbacks
final_history = final_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*60)
print("FINAL MODEL TRAINING COMPLETE")
print("="*60)
print(f"✓ Final model trained!")
print(f"  - Total epochs: {len(final_history.history['loss'])}")
print(f"  - Best validation loss: {min(final_history.history['val_loss']):.4f}")
print(f"  - Best validation accuracy: {max(final_history.history['val_accuracy']):.4f}")



In [ ]:
# Step 7.4: Evaluate Final Model on Test Set
# ============================================================

# Load best final model
final_best_model = load_model('../models/heart_failure_model_best.h5')

print("="*60)
print("EVALUATING FINAL MODEL (Test Set)")
print("="*60)

# Evaluate on test set
test_results = final_best_model.evaluate(X_test, y_test, verbose=0)

print("📊 Final Model Performance (Test Set):")
print(f"  - Loss: {test_results[0]:.4f}")
print(f"  - Accuracy: {test_results[1]:.4f} ({test_results[1]*100:.2f}%)")
print(f"  - Precision: {test_results[2]:.4f}")
print(f"  - Recall: {test_results[3]:.4f}")

# Calculate F1 score
final_f1 = 2 * (test_results[2] * test_results[3]) / (test_results[2] + test_results[3])
print(f"  - F1 Score: {final_f1:.4f}")

# Get predictions
y_pred_proba_final = final_best_model.predict(X_test, verbose=0)
y_pred_final = (y_pred_proba_final > 0.5).astype(int).flatten()

# Confusion Matrix
cm_final = confusion_matrix(y_test, y_pred_final)
print("\n📊 Confusion Matrix:")
print(cm_final)

print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred_final, target_names=['No Disease', 'Disease']))

# Compare with initial model
print("\n📊 Performance Comparison:")
print(f"  Initial Model Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"  Final Model Accuracy:   {test_results[1]:.4f} ({test_results[1]*100:.2f}%)")
improvement = test_results[1] - test_accuracy
if improvement > 0:
    print(f"  ✓ Improvement: +{improvement:.4f} (+{improvement*100:.2f}%)")
elif improvement < 0:
    print(f"  ⚠ Change: {improvement:.4f} ({improvement*100:.2f}%)")
else:
    print(f"  → No change")



In [ ]:
# Step 7.5: Save Final Model
# ============================================================

import pickle

print("="*60)
print("SAVING FINAL MODEL")
print("="*60)

# Save final model
final_best_model.save('../models/heart_failure_model.h5')
print("✓ Final model saved to '../models/heart_failure_model.h5'")

# Also save scaler (needed for predictions)
with open('../models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print("✓ Scaler saved to '../models/scaler.pkl'")

print("\n" + "="*60)
print("✅ PHASE 7 COMPLETE!")
print("="*60)
print("✓ Model optimized and saved")
print("✓ Ready for Person C (Evaluation & Reporting)")
print("\n📦 Deliverables:")
print("  - models/heart_failure_model.h5 (final model)")
print("  - models/scaler.pkl (preprocessing scaler)")
print("  - Test set performance metrics")
print("="*60)

